![paleoCAMP logo](./images/paleoCAMPLogo.png)

# 3. (Optional) Run a pair of piControl and midHolocene simulations

**Tutorials at [the 2025 paleoCAMP](https://paleoclimate.camp/) | June 16–June 30, 2025**
       
[Jiang Zhu](https://staff.cgd.ucar.edu/jiangzhu/)  
[jiangzhu@ucar.edu](mailto:jiangzhu@ucar.edu)  
Climate & Global Dynamics Laboratory  
NSF National Center for Atmospheric Research   

------

**Learning Objectives:**

* Run simple CESM simulations
* Gain hands-on skills on customizing the run length and namelists
* Know the importance of validating your numerical experiments 
* Understand the idea of more complex simulation by examining steps to set up an LGM simulation
  
**Time to learn**: 60 minutes

---
**How to get started?**

**Use terminal from your system directly**
```bash
ssh -XY YOUR_SUERNAME@derecho.hpc.ucar.edu
```
**Type in your CIT password (the one you set up with CISL) and approve the DUO push**

<!-- <p align="center"><img src="./images/derecho_ssh_login.png" width="500"/></p> -->

![Derecho SSH](./images/derecho_ssh_login.png)    

---
<div class="alert alert-info">   

**REMINDER: CESM directories**

- Source Code Directory ("Headquarter"): `/glade/work/jiangzhu/paleocamp/cesm2.1.5/`  
- Case Directory ("Production Control Room"): e.g., `~/b.e21.B1850.f19_g17.piControl.001`  
- Run directory ("Factory"): e.g., `/glade/derecho/scratch/jiangzhu/b.e21.B1850.f19_g17.piControl.001/run`
- Archive directory ("Storage"): `/glade/derecho/scratch/jiangzhu/archive/b.e21.B1850.f19_g17.piControl.001/`
</div>

---
## Task 1: run a fully coupled preindustrial simulation for one year

### Hints:  
- What are the four steps?  
- What is the compset for fully coupled preindustrial?  
- Use resolution of `f19_g17` for fast throughput
- Following the [CESM naming conventions](https://www.cesm.ucar.edu/models/cesm2/naming-conventions), use `b.e21.B1850.f19_g17.piControl.001` as the case name
- Which XML variable should you change to tell the model to run for one year? (do this before `./case.submit`)
- It may take ~16 minutes to finish the one-year simulation.

<div class="alert alert-success">   
<details>
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>
 

**1. create a new case:** go into the `scripts` directory in the CESM Code and run the `create_newcase` command
```bash
cd /glade/work/jiangzhu/paleocamp/cesm2.1.5/cime/scripts
./create_newcase --case ~/b.e21.B1850.f19_g17.piControl.001 --compset B1850 --res f19_g17 --project UAZN0042
```
<br>

**2. case setup:** go into the Case Directory and run `case.setup`
```bash
cd ~/b.e21.B1850.f19_g17.piControl.001
./case.setup
```
<br>   

**3. case build:** stay in the Case Directory and run `case.build` (change the run length using `xmlchange`)
```bash
./xmlchange STOP_OPTION=nyears,STOP_N=1
./case.build
```
<br>

**4. Build and submit**
```bash
./case.submit
```

</details>
</div>

### Further information
- Building the executable may take more than 10 minutes.
- While you wait, open a second terminal and practice your Unix commands (such as `cd`, `ls`, and `grep`) by visiting your directories of "Production Control Room", "Factory", and "Headquarter", and see what are there.
- After submitting the job, use `qstat -u YOUR_USERNAME` to check the status of your job.
- Once the model is running, select one of your favorite components and read the log file (in the Run Directory, "_Factory_") and see what the model is doing.

---
## Task 2: run a fully coupled mid-Holocene simulation with the orbital forcing for one year

### Hints:  
- Use compset `B1850` and edit `user_nl_cpl` to set up the orbital configuration (no mid-Holocene compset available)
- Following the [CESM naming conventions](https://www.cesm.ucar.edu/models/cesm2/naming-conventions), use `b.e21.B1850.f19_g17.midHolocene.001` as the case name
- According to Table 1 of [Otto-Bliesner et al., (2017)](https://gmd.copernicus.org/articles/10/3979/2017/gmd-10-3979-2017.pdf), Eccentricity = 0.018682, Obliquity (degrees) = 24.105, Perihelion = 0.87 (for simplicity, we don't consider the other forcings here, i.e., CO2)
  - In the CESM world, this means
    - orb_mode = "fixed_parameters"
    - orb_eccen = 0.018682
    - orb_obliq = 24.105
    - orb_mvelp = 0.87
  - Search `orb_` [here](https://docs.cesm.ucar.edu/models/cesm2/settings/current/drv_nml.html) to learn more
- Use Unix editor `vi`/`emacs`/`nano` to open, edit, and save
- These namelist changes can be done after `case.build` but before `case.submit`


<div class="alert alert-success">   
<details>
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>
 

**1. create a new case:** go into scripts directory in the CESM Code and run the `create_newcase` command
```bash
cd /glade/work/jiangzhu/paleocamp/cesm2.1.5/cime/scripts
./create_newcase --case ~/b.e21.B1850.f19_g17.midHolocene.001 --compset B1850 --res f19_g17  --project UAZN0042
```
<br>

**2. case setup:** go into the Case Directory and run `case.setup`
```bash
cd ~/b.e21.B1850.f19_g17.midHolocene.001
./case.setup
```
<br>   

**3. case build:** stay in the Case Directory and run `case.build`
- Add the following to user_nl_cpl
```bash
 orb_mode = "fixed_parameters"
 orb_eccen = 0.018682
 orb_obliq = 24.105
 orb_mvelp = 0.87
```
- Change run length and build the case
```bash
./xmlchange STOP_OPTION=nyears,STOP_N=1
./case.build
```
<br>

**4. Build and submit**
```bash
./case.submit
```

</details>
</div>

---
## Task 3: how to validate your simulation setup

### Hints:  
- Going through the log files (in the RUN directory, aka "Factory") is one way to see how the model is being run.
- Examining `*_in` files (e.g., `CasesDocs/drv_in` in the Case Directory) is another way to check the setup.

<div class="alert alert-success">   
<details>
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>
    
1. If you want to check the log file, `cpl.log.xxx`, in the Run Directory (when model is still running) or in your Storage Directory (when the simulation and archiving have finished).
```bash
vi /glade/derecho/scratch/jiangzhu/archive/b.e21.B1850.f19_g17.midHolocene.001/logs/cpl.log.3950708.desched1.240330-011322.gz
```
type `/orb_params` to search, you should see the following
```
 (shr_orb_params) Calculate characteristics of the orbit:
 (shr_orb_params) Calculate orbit for year:     -4050
 (shr_orb_params) ------ Computed Orbital Parameters ------
 (shr_orb_params) Eccentricity      =   1.868182E-02
 (shr_orb_params) Obliquity (deg)   =   2.410538E+01
 (shr_orb_params) Obliquity (rad)   =   4.207183E-01
 (shr_orb_params) Long of perh(deg) =   8.696128E-01
 (shr_orb_params) Long of perh(rad) =   3.156770E+00
 (shr_orb_params) Long at v.e.(rad) =  -5.751115E-04
```
Try also `grep shr_orb_params YOUR_cpl.log`

2. If you want to check the `drv_in` file
```bash
vi /glade/u/home/jiangzhu/b.e21.B1850.f19_g17.midHolocene.001/CaseDocs/drv_in
```
type `/orb_` to search, you should see the following
```
   orb_eccen = 0.018682
   orb_mode = "fixed_parameters"
   orb_mvelp = 0.87
   orb_obliq = 24.105
```

Try also `grep orb_ CaseDocs/drv_in` in the Case Directory

</details>
</div>

---
## Summary
- Set up and run a CESM simulation in four steps
- Customize the simulation with `xmlchange` and namelist change